In [1]:
import pandas as pd
import MySQLdb
df = pd.read_csv('../data/원본데이터/행정안전부_지역별(행정동) 성별 연령별 주민등록 인구수_20230430.csv', encoding='CP949')
df.head()

,행정기관코드,기준연월,시도명,시군구명,읍면동명,계,남자,여자,만0세남자,만1세남자,...,만101세여자,만102세여자,만103세여자,만104세여자,만105세여자,만106세여자,만107세여자,만108세여자,만109세여자,만110세이상여자
0,1111051500,2023-04-30,서울특별시,종로구,청운효자동,11531,5280,6251,17,21,...,0,1,0,0,0,0,0,0,0,0
1,1111053000,2023-04-30,서울특별시,종로구,사직동,9104,4047,5057,13,19,...,0,0,0,0,0,0,0,0,0,0
2,1111054000,2023-04-30,서울특별시,종로구,삼청동,2301,1096,1205,1,2,...,0,0,0,0,0,0,0,0,0,0
3,1111055000,2023-04-30,서울특별시,종로구,부암동,9168,4334,4834,10,14,...,1,1,0,0,0,0,0,0,0,0
4,1111056000,2023-04-30,서울특별시,종로구,평창동,17520,8184,9336,28,43,...,2,0,1,0,0,1,0,1,0,0


In [2]:
df = df[['시도명', '시군구명', '읍면동명']]

In [3]:
df.dropna(inplace=True)

In [4]:
df['읍면동명'] = df['읍면동명'].apply(lambda x : x.replace('.',''))

In [5]:
for town in df['읍면동명'].drop_duplicates():
    try:
        del globals()[f'df_{town}']
    except:
        pass

In [6]:
for town in df['읍면동명'].drop_duplicates():
    n = df[df['읍면동명'] == town]['읍면동명'].values[0]
    globals()[f'df_{town}'] = pd.DataFrame({'읍면동명': [n,n,n,n,n,n], 
                                            '어트랙션': ['여행지','볼거리/놀거리/명소','취미생활','맛집','카페','아이와함께']})

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query='
driver.get(url)
for town in df['읍면동명'].drop_duplicates():
    column = []
    for attr in ['여행지','놀거리','취미생활','맛집','카페','아이와함께']:
        keyword = town + ' ' + attr
        driver.find_element(By.CLASS_NAME, 'box_window').send_keys(keyword)
        driver.find_element(By.CLASS_NAME, 'bt_search').click()
        time.sleep(0.25)
        driver.find_element(By.CLASS_NAME, 'box_window').clear()
        lst = []
        lists = driver.find_elements(By.CLASS_NAME, 'place_bluelink')
        for l in lists:
            try:
                lst.append(l.text)
            except:
                lst.append('')
        column.append(', '.join(lst))
    globals()[f'df_{town}']['어트랙션_목록'] = column
driver.close()

KeyboardInterrupt: 

In [8]:
total = pd.DataFrame()
for town in df['읍면동명'].drop_duplicates():
    total = pd.concat([total, globals()[f'df_{town}']])
total.head()

,읍면동명,어트랙션,어트랙션_목록
0,청운효자동,여행지,"경복궁, 수성동계곡, 광화문광장, 서촌마을, 세종대왕동상, 광화문"
1,청운효자동,볼거리/놀거리/명소,"대한민국역사박물관, 세종문화회관, 광화문광장, 국립고궁박물관, 윤동주문학관, 경복궁"
2,청운효자동,취미생활,"로라홈, 커스텀테라피, 가회민화박물관, 동림매듭공방, 꽃자리자수공방, 틈 틈틈이 하..."
3,청운효자동,맛집,"야끼니꾸소량 경복궁점, 도취, 잘빠진메밀 서촌 본점, 비스트로 친친, 라파리나, 스..."
4,청운효자동,카페,"서촌금상고로케, 호전다실, 스펙터, 더숲 초소책방, 서촌스코프, 아키비스트 서촌, ..."


In [9]:
df = pd.merge(df, total, on='읍면동명', how='left')
df.head()

,시도명,시군구명,읍면동명,어트랙션,어트랙션_목록
0,서울특별시,종로구,청운효자동,여행지,"경복궁, 수성동계곡, 광화문광장, 서촌마을, 세종대왕동상, 광화문"
1,서울특별시,종로구,청운효자동,볼거리/놀거리/명소,"대한민국역사박물관, 세종문화회관, 광화문광장, 국립고궁박물관, 윤동주문학관, 경복궁"
2,서울특별시,종로구,청운효자동,취미생활,"로라홈, 커스텀테라피, 가회민화박물관, 동림매듭공방, 꽃자리자수공방, 틈 틈틈이 하..."
3,서울특별시,종로구,청운효자동,맛집,"야끼니꾸소량 경복궁점, 도취, 잘빠진메밀 서촌 본점, 비스트로 친친, 라파리나, 스..."
4,서울특별시,종로구,청운효자동,카페,"서촌금상고로케, 호전다실, 스펙터, 더숲 초소책방, 서촌스코프, 아키비스트 서촌, ..."


In [10]:
df.fillna('', inplace=True)

In [5]:
cnx = MySQLdb.connect(user='web', passwd='1234',
                      host='localhost', db='travel')

cursor = cnx.cursor()
table_name = 'total_attraction'

create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(['{} VARCHAR(255)'.format(col) for col in df.columns])})"
cursor.execute(create_table_query)

0

In [6]:
for row in df.itertuples():
    insert_query = f"INSERT INTO {table_name} VALUES {tuple(row[1:])}"
    cursor.execute(insert_query)

In [7]:
cnx.commit()

In [8]:
cursor.close()
cnx.close()